<h1><center>Regression Model, using Spark MLib to predict Mobile price based on various features</center></h1>
<hr><hr><hr>

## Configurations:
---------------------

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName("Phone Price Prediction").getOrCreate()

In [4]:
spark

In [5]:
sc = spark.sparkContext

In [6]:
spark.conf.set("spark.sql.shuffle.partitions", sc.defaultParallelism)

In [7]:
import pyspark.sql.functions as F
from pyspark.sql.types import IntegerType, ArrayType, DoubleType

In [8]:
mobile_df = spark.read.csv("./data/mobile phone price prediction.csv", header=True, inferSchema=True).withColumnRenamed("_c0", "id")
mobile_df.show()

+---+--------------------+------+----------+--------------------+--------+-----------------+----------+--------------------+--------------------+---------------+------+-------+---------------+------------------+--------------------+--------------------+-------------------+
| id|                Name|Rating|Spec_score|           No_of_sim|     Ram|          Battery|   Display|              Camera|     External_Memory|Android_version| Price|company| Inbuilt_memory|     fast_charging|   Screen_resolution|           Processor|     Processor_name|
+---+--------------------+------+----------+--------------------+--------+-----------------+----------+--------------------+--------------------+---------------+------+-------+---------------+------------------+--------------------+--------------------+-------------------+
|  0|Samsung Galaxy F1...|  4.65|        68|Dual Sim, 3G, 4G,...|4 GB RAM|6000 mAh Battery |6.6 inches|50 MP + 2 MP Dual...|Memory Card Suppo...|             13| 9,999|Samsung| 1

In [9]:
mobile_df.count()

1370

In [74]:
print( type(mobile_df.toPandas()) )
print( mobile_df.toPandas() )

<class 'pandas.core.frame.DataFrame'>
        id                                  Name  Rating  Spec_score  \
0        0                 Samsung Galaxy F14 5G    4.65          68   
1        1                    Samsung Galaxy A11    4.20          63   
2        2                    Samsung Galaxy A13    4.30          75   
3        3                    Samsung Galaxy F23    4.10          73   
4        4  Samsung Galaxy A03s (4GB RAM + 64GB)    4.10          69   
...    ...                                   ...     ...         ...   
1365  1365                               TCL 40R    4.05          75   
1366  1366                 TCL 50 XL NxtPaper 5G    4.10          80   
1367  1367                 TCL 50 XE NxtPaper 5G    4.00          80   
1368  1368                    TCL 40 NxtPaper 5G    4.50          79   
1369  1369                           TCL Trifold    4.65          93   

                                No_of_sim        Ram            Battery  \
0           Dual Sim, 

In [10]:
opt = {
    "header": True,
    "inferSchema":True
}

mobile_df2 = spark.read.format("csv").options(**opt).load("./data/mobile phone price prediction.csv")
mobile_df2.show()

+---+--------------------+------+----------+--------------------+--------+-----------------+----------+--------------------+--------------------+---------------+------+-------+---------------+------------------+--------------------+--------------------+-------------------+
|_c0|                Name|Rating|Spec_score|           No_of_sim|     Ram|          Battery|   Display|              Camera|     External_Memory|Android_version| Price|company| Inbuilt_memory|     fast_charging|   Screen_resolution|           Processor|     Processor_name|
+---+--------------------+------+----------+--------------------+--------+-----------------+----------+--------------------+--------------------+---------------+------+-------+---------------+------------------+--------------------+--------------------+-------------------+
|  0|Samsung Galaxy F1...|  4.65|        68|Dual Sim, 3G, 4G,...|4 GB RAM|6000 mAh Battery |6.6 inches|50 MP + 2 MP Dual...|Memory Card Suppo...|             13| 9,999|Samsung| 1

In [11]:
mobile_df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Rating: double (nullable = true)
 |-- Spec_score: integer (nullable = true)
 |-- No_of_sim: string (nullable = true)
 |-- Ram: string (nullable = true)
 |-- Battery: string (nullable = true)
 |-- Display: string (nullable = true)
 |-- Camera: string (nullable = true)
 |-- External_Memory: string (nullable = true)
 |-- Android_version: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- company: string (nullable = true)
 |-- Inbuilt_memory: string (nullable = true)
 |-- fast_charging: string (nullable = true)
 |-- Screen_resolution: string (nullable = true)
 |-- Processor: string (nullable = true)
 |-- Processor_name: string (nullable = true)



In [12]:
(
    mobile_df.select("Processor_name").distinct()
    .show(truncate=False)
)

+-------------------+
|Processor_name     |
+-------------------+
|Snapdragon 695     |
|Snapdragon 750G    |
|Snapdragon 7s Gen2 |
|Exynos 2100        |
|Dimensity 6020     |
|Snapdragon 7 Gen3  |
|Snapdragon 782G    |
|Dimensity 8300     |
|Dimensity 9300 Plus|
|Snapdragon  7 Gen1 |
|Snapdragon 7+ Gen3 |
|Dimensity 8100-Max |
|Snapdragon  870    |
|Dimensity 8100     |
|Snapdragon  480+   |
|Tensor G2          |
|Snapdragon  765G   |
|Snapdragon  439    |
|Exynos 1280        |
|Exynos 2400        |
+-------------------+
only showing top 20 rows



In [13]:
mobile_df_req = mobile_df.select([
    "Rating", 
    "Spec_score", 
    "Ram", 
    "Battery", 
    "Display", 
    "Camera", 
    "Android_version", 
    "Inbuilt_memory",
    "fast_charging",
    "Screen_resolution",
    "Price"
])
mobile_df_req.show(truncate=False)

+------+----------+--------+-----------------+----------+---------------------------------------------------------+---------------+---------------+------------------+---------------------------------------------+------+
|Rating|Spec_score|Ram     |Battery          |Display   |Camera                                                   |Android_version|Inbuilt_memory |fast_charging     |Screen_resolution                            |Price |
+------+----------+--------+-----------------+----------+---------------------------------------------------------+---------------+---------------+------------------+---------------------------------------------+------+
|4.65  |68        |4 GB RAM|6000 mAh Battery |6.6 inches|50 MP + 2 MP Dual Rear &amp; 13 MP Front Camera          |13             | 128 GB inbuilt| 25W Fast Charging| 2408 x 1080 px Display with Water Drop Notch|9,999 |
|4.2   |63        |2 GB RAM|4000 mAh Battery |6.4 inches|13 MP + 5 MP + 2 MP Triple Rear &amp; 8 MP Front Camera  |10   

In [14]:
# Filtering all those mobile phones that have more than 2 cameras in it.
(
    mobile_df_req
    .select("Camera")
    .withColumn("Cam", F.regexp_replace(F.col("Camera"), r'([^0-9])+' , ',' ) )
    .withColumn("Cam2",F.expr("substring(Cam, 1, length(Cam) - 1)"))
    .withColumn("cam3",F.split("Cam2",','))
    .filter(F.size(F.col("cam3"))>2)
    .show(truncate=False)
)

+---------------------------------------------------------+-----------+----------+---------------+
|Camera                                                   |Cam        |Cam2      |cam3           |
+---------------------------------------------------------+-----------+----------+---------------+
|50 MP + 2 MP Dual Rear &amp; 13 MP Front Camera          |50,2,13,   |50,2,13   |[50, 2, 13]    |
|13 MP + 5 MP + 2 MP Triple Rear &amp; 8 MP Front Camera  |13,5,2,8,  |13,5,2,8  |[13, 5, 2, 8]  |
|13 MP + 2 MP + 2 MP Triple Rear &amp; 5 MP Front Camera  |13,2,2,5,  |13,2,2,5  |[13, 2, 2, 5]  |
|50 MP + 2 MP Dual Rear &amp; 5 MP Front Camera           |50,2,5,    |50,2,5    |[50, 2, 5]     |
|48 MP + 8 MP + 5 MP Triple Rear &amp; 20 MP Front Camera |48,8,5,20, |48,8,5,20 |[48, 8, 5, 20] |
|50 MP + 2 MP + 2 MP Triple Rear &amp; 13 MP Front Camera |50,2,2,13, |50,2,2,13 |[50, 2, 2, 13] |
|50 MP + 5 MP + 2 MP Triple Rear &amp; 8 MP Front Camera  |50,5,2,8,  |50,5,2,8  |[50, 5, 2, 8]  |
|50 MP + 8

In [15]:
(
    mobile_df_req
    .select("Screen_resolution")
    .withColumn( "Screen_resolution2", F.regexp_replace( F.col("Screen_resolution"), r'([^0-9])+' , ',' ) )  
    .where( F.col("Screen_resolution2") != F.lit(",") )
    .orderBy( F.col("Screen_resolution2") )
    .show(truncate=False)
)

+---------------------------------------------+------------------+
|Screen_resolution                            |Screen_resolution2|
+---------------------------------------------+------------------+
| 1080 x 1920 px Display                      |,1080,1920,       |
| 1080 x 1920 px                              |,1080,1920,       |
| 1080 x 2160 px                              |,1080,2160,       |
| 1080 x 2220 px Display                      |,1080,2220,       |
| 1080 x 2246 px Display with Large Notch     |,1080,2246,       |
| 1080 x 2256 px Display                      |,1080,2256,       |
| 1080 x 2280 px Display                      |,1080,2280,       |
| 1080 x 2280 px Display with Water Drop Notch|,1080,2280,       |
| 1080 x 2340 px                              |,1080,2340,       |
| 1080 x 2340 px                              |,1080,2340,       |
| 1080 x 2340 px Display with Punch Hole      |,1080,2340,       |
| 1080 x 2340 px                              |,1080,2340,    

## - Feature Columns Generation: 
-------------------------------------
### Formatting required columns to Numerical datatype, by Data Extraction and Type Conversion:

In [16]:
mobile_df_req_final = (
    mobile_df_req.withColumn("Price",F.regexp_replace(F.col("Price"),',',''))  # Removing the comma, to convert string to integer
    .withColumn( "Price", F.col("Price").cast(IntegerType()) )
    .withColumn( "Ram", F.regexp_extract( F.col("Ram"), r'([0-9])+', 0 ) )  # Taking the numerical part of RAM column data
    .withColumn( "Ram", F.col("Ram").cast("int") )
    .withColumn( "Battery", F.regexp_extract( F.col("Battery"), r'([0-9])+', 0 ) )  # Taking the numerical part from Battery column data.
    .withColumn( "Battery", F.col("Battery").cast(IntegerType()) )
    .withColumn( "Display", F.regexp_extract( F.col("Display"), r'([0-9].[0-9])', 1 ) )  # Taking only the numerical part of display
    .withColumn( "Display", F.col("Display").cast("float") )
    .withColumn( "Camera", F.regexp_replace( F.col("Camera"), r'([^0-9])+' , ',' ) )  # Replacing all the non-numeric characters with comma
    .where( F.col("Camera") != F.lit(",") )  # Filtering out all the records having no number in Camera column.
    .withColumn( "Camera", F.expr("substring(Camera, 1, length(Camera)-1)") )  # Removing extra comma(,) at the end
    .withColumn( "Camera", F.split(F.col("Camera"), ",") )  # Converting the comma-separated string into an array-type column.
    .withColumn( "Camera", F.col("Camera").cast(ArrayType(IntegerType())) )  # Converting array of strings to an array of integers.
    .withColumn( "Android_version", F.col("Android_version").cast(IntegerType()) )
    .withColumn( "Inbuilt_memory", F.regexp_extract(F.col("Inbuilt_memory"), r"([0-9]+)", 1 ) )  # Extracting numerical part of Inbuilt_Memory
    .withColumn( "Inbuilt_memory", F.col("Inbuilt_memory").cast("int") )
    .withColumn("fast_charging", F.when(F.col("fast_charging").isNull(), 0).otherwise(F.col("fast_charging")) )  # Replacing null value with 0
    .withColumn("fast_charging", F.regexp_extract( F.col("fast_charging"), r"([0-9]+)", 1 ) )  # Extracting numerical part of fast_charging
    .withColumn("fast_charging", F.col("fast_charging").cast("int") )
    .withColumn("Screen_resolution", F.regexp_replace( F.col("Screen_resolution"), r"([^0-9])+", "," ) )  # Replacing non-numeric parts with comma(,)
    .where( F.col("Screen_resolution") != F.lit(",") )  # Filtering out all the records having no number in Camera column.
    .withColumn("Screen_resolution", F.expr("substring(Screen_resolution, 2, len(Screen_resolution)-2)") )  # Removing comma at beginning and at end.
    .withColumn("Screen_resolution", F.split(F.col("Screen_resolution"), ",") )  #  Converting comma-separated string to an array
    .withColumn("Screen_resolution", F.col("Screen_resolution").cast(ArrayType(IntegerType())) )
)

In [17]:
mobile_df_req_final.printSchema()

root
 |-- Rating: double (nullable = true)
 |-- Spec_score: integer (nullable = true)
 |-- Ram: integer (nullable = true)
 |-- Battery: integer (nullable = true)
 |-- Display: float (nullable = true)
 |-- Camera: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- Android_version: integer (nullable = true)
 |-- Inbuilt_memory: integer (nullable = true)
 |-- fast_charging: integer (nullable = true)
 |-- Screen_resolution: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- Price: integer (nullable = true)



In [18]:
mobile_df_req_final.show()

+------+----------+---+-------+-------+---------------+---------------+--------------+-------------+-----------------+-----+
|Rating|Spec_score|Ram|Battery|Display|         Camera|Android_version|Inbuilt_memory|fast_charging|Screen_resolution|Price|
+------+----------+---+-------+-------+---------------+---------------+--------------+-------------+-----------------+-----+
|  4.65|        68|  4|   6000|    6.6|    [50, 2, 13]|             13|           128|           25|     [2408, 1080]| 9999|
|   4.2|        63|  2|   4000|    6.4|  [13, 5, 2, 8]|             10|            32|           15|      [720, 1560]| 9990|
|   4.3|        75|  4|   5000|    6.6|        [50, 8]|             12|            64|           25|     [1080, 2408]|11999|
|   4.1|        73|  4|   6000|    6.4|       [48, 13]|             12|            64|            0|      [720, 1600]|11999|
|   4.1|        69|  4|   5000|    6.5|  [13, 2, 2, 5]|             11|            64|           15|      [720, 1600]|11999|


## - Handling Outliers and Null values:
-----------------------------------------
Removing the columns with Outliers and Null values.

In [19]:
mobile_df_req_final.orderBy( F.col("Camera").asc() ).show(50, truncate=False)

+------+----------+---+-------+-------+---------------+---------------+--------------+-------------+-----------------+-----+
|Rating|Spec_score|Ram|Battery|Display|Camera         |Android_version|Inbuilt_memory|fast_charging|Screen_resolution|Price|
+------+----------+---+-------+-------+---------------+---------------+--------------+-------------+-----------------+-----+
|4.0   |42        |1  |1900   |4.5    |[5, 2]         |null           |16            |0            |[480, 854]       |7499 |
|4.25  |55        |2  |3000   |null   |[5, 2]         |14             |32            |10           |[540, 1092]      |7990 |
|4.05  |69        |6  |5000   |6.7    |[8, 0, 8, 5]   |14             |128           |18           |[720, 1650]      |6999 |
|4.45  |66        |4  |5000   |6.5    |[8, 0, 8, 5]   |13             |64            |10           |[720, 1600]      |8499 |
|4.75  |70        |6  |5000   |6.5    |[8, 0, 8, 5]   |13             |128           |10           |[720, 1600]      |8999 |


- `Rating` column has no null values and no outliers.
- `Spec_score` column has no null values and no outliers.
- `Ram` column has upper-limit exceeding outliers. Idle Range: `1 to 30`
- `Battery` column has outliers below lower-limit. Idle Range: `500 to 11000`
- `Display` column `has null values`, and has outliern values below and above lower-limit and upper limit respec. Idle Range: `4.0 to 8.5`
- `Camera` column has arrays with `null` value only. Array size to be considered: `1 to 4`. More than 4 value containing arrays should be dropped.
- `Android_version` column has more than `400 null-value` containing records.
- `Inbuilt_memory` column `has null values`, as well as outlier values before lower-limit. Idle Range: `2 to 512`
- `fast_charging` column `has null values`.
- `Screen_resolution` column `has null values`, and array with null value only.

In [20]:
features_df = (
    mobile_df_req_final.where( F.col("Ram") <= 30 )
    .where( F.col("Battery") >= 500 )
    .where( ( F.col("Display").isNotNull() ) & ( F.col("Display") >= 4.0 ) & ( F.col("Display") <= 8.5 ) )
    .where( ( F.col("Camera").isNotNull() ) & ( F.size(F.col("Camera")) <= 4 ) )
    .where( F.col("Android_version").isNotNull() )
    .where( ( F.col("Inbuilt_memory").isNotNull() ) & ( F.col("Inbuilt_memory") >= 2 ) )
    .where( F.col("fast_charging").isNotNull() )
    .where( F.col("Screen_resolution").isNotNull() )
)

In [21]:
features_df.count()

819

In [39]:
features_df.orderBy( F.col("Screen_resolution").asc() ).show()

+------+----------+---+-------+-------+-------------+---------------+--------------+-------------+-----------------+-----+
|Rating|Spec_score|Ram|Battery|Display|       Camera|Android_version|Inbuilt_memory|fast_charging|Screen_resolution|Price|
+------+----------+---+-------+-------+-------------+---------------+--------------+-------------+-----------------+-----+
|  4.45|        61|  2|   3900|    6.2|[13, 5, 2, 8]|             10|            32|            0|      [720, 1520]| 9499|
|   4.7|        67|  6|   4000|    6.2|     [16, 13]|             10|           128|            0|      [720, 1520]| 7499|
|   4.3|        63|  6|   4000|    6.5|      [13, 5]|             12|           128|           10|      [720, 1520]|13990|
|  4.65|        65|  4|   5000|    6.5|  [13, 2, 13]|             10|            64|            0|      [720, 1544]| 9990|
|   4.2|        63|  2|   4000|    6.4|[13, 5, 2, 8]|             10|            32|           15|      [720, 1560]| 9990|
|   4.1|        

In [23]:
features_df.select( "Camera" ).orderBy( F.col("Camera") ).show()

+------------+
|      Camera|
+------------+
|[8, 0, 8, 5]|
|[8, 0, 8, 5]|
|[8, 0, 8, 5]|
|[8, 0, 8, 5]|
|      [8, 5]|
|      [8, 5]|
|      [8, 5]|
|      [8, 5]|
|      [8, 5]|
|      [8, 5]|
|      [8, 5]|
|      [8, 5]|
|      [8, 5]|
|      [8, 5]|
|      [8, 5]|
|      [8, 5]|
|      [8, 5]|
|      [8, 5]|
|      [8, 5]|
|      [8, 5]|
+------------+
only showing top 20 rows



## Handling Array Type columns:
---------------------------------
- To apply Regression modelling technique, input and output columns type should of integer or float type, else it cannot be transformed into a Vector using the `VectorAssembler` class. Thus, array type columns must be represented in numerical format.
- In our case, we have `Camera` and `Screen_resolution` columns as array type columns containing integer values. To represent their true nature, and value of magnitude, we can transform them in the following way:
    - The camera column can be transformed to an integer column by adding all the integers in the array, as this would roughly show the total number of megapixels all the camera have, when combined.
    - The screen resulotion column could be transformed to an integer by multiplying the integers, as this would depict the total number of pixels the mobile has in its display.

In [40]:
@F.udf( returnType=IntegerType() )
def add_array_elements( arr ):
    result = 0
    for i in arr:
        result += i
    return result

@F.udf( returnType=IntegerType() )
def multiply_array_elements( arr ):
    result = 1
    for i in arr:
        result *= i
    return result

In [42]:
features_df = (
    features_df.withColumn( "Camera", add_array_elements(F.col("Camera")) )
    .withColumn( "Screen_resolution", multiply_array_elements( F.col("Screen_resolution") ) )
)

In [30]:
features_df.write.parquet("file:///data/mobile_features_cleaned.parquet", mode="overwrite")

Py4JJavaError: An error occurred while calling o286.parquet.
: ExitCodeException exitCode=-1073741515: 
	at org.apache.hadoop.util.Shell.runCommand(Shell.java:1007)
	at org.apache.hadoop.util.Shell.run(Shell.java:900)
	at org.apache.hadoop.util.Shell$ShellCommandExecutor.execute(Shell.java:1212)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1306)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1288)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.R

In [43]:
features_df.show()

+------+----------+---+-------+-------+------+---------------+--------------+-------------+-----------------+-----+
|Rating|Spec_score|Ram|Battery|Display|Camera|Android_version|Inbuilt_memory|fast_charging|Screen_resolution|Price|
+------+----------+---+-------+-------+------+---------------+--------------+-------------+-----------------+-----+
|  4.65|        68|  4|   6000|    6.6|    65|             13|           128|           25|          2600640| 9999|
|   4.2|        63|  2|   4000|    6.4|    28|             10|            32|           15|          1123200| 9990|
|   4.3|        75|  4|   5000|    6.6|    58|             12|            64|           25|          2600640|11999|
|   4.1|        73|  4|   6000|    6.4|    61|             12|            64|            0|          1152000|11999|
|   4.1|        69|  4|   5000|    6.5|    22|             11|            64|           15|          1152000|11999|
|   4.4|        75|  6|   5000|    6.5|    57|             12|          

In [44]:
features_df.printSchema()

root
 |-- Rating: double (nullable = true)
 |-- Spec_score: integer (nullable = true)
 |-- Ram: integer (nullable = true)
 |-- Battery: integer (nullable = true)
 |-- Display: float (nullable = true)
 |-- Camera: integer (nullable = true)
 |-- Android_version: integer (nullable = true)
 |-- Inbuilt_memory: integer (nullable = true)
 |-- fast_charging: integer (nullable = true)
 |-- Screen_resolution: integer (nullable = true)
 |-- Price: integer (nullable = true)



# Creating the Regression Model:
----------------------------------

In [35]:
from pyspark.ml.feature import VectorAssembler

In [37]:
input_features_assembler = VectorAssembler(
    inputCols = [
        'Rating',
        'Spec_score',
        'Ram',
        'Battery',
        'Display',
        'Camera',
        'Android_version',
        'Inbuilt_memory',
        'fast_charging',
        'Screen_resolution'
    ],
    outputCol = "independent_features"
)

In [45]:
features_df_final = input_features_assembler.transform( features_df )
features_df_final.show()

+------+----------+---+-------+-------+------+---------------+--------------+-------------+-----------------+-----+--------------------+
|Rating|Spec_score|Ram|Battery|Display|Camera|Android_version|Inbuilt_memory|fast_charging|Screen_resolution|Price|independent_features|
+------+----------+---+-------+-------+------+---------------+--------------+-------------+-----------------+-----+--------------------+
|  4.65|        68|  4|   6000|    6.6|    65|             13|           128|           25|          2600640| 9999|[4.65,68.0,4.0,60...|
|   4.2|        63|  2|   4000|    6.4|    28|             10|            32|           15|          1123200| 9990|[4.2,63.0,2.0,400...|
|   4.3|        75|  4|   5000|    6.6|    58|             12|            64|           25|          2600640|11999|[4.3,75.0,4.0,500...|
|   4.1|        73|  4|   6000|    6.4|    61|             12|            64|            0|          1152000|11999|[4.1,73.0,4.0,600...|
|   4.1|        69|  4|   5000|    6.5|  

In [48]:
features_df_final = features_df_final.select( ["independent_features", "Price"] )

In [50]:
features_df_final.show(truncate=False)

+-----------------------------------------------------------------------+-----+
|independent_features                                                   |Price|
+-----------------------------------------------------------------------+-----+
|[4.65,68.0,4.0,6000.0,6.599999904632568,65.0,13.0,128.0,25.0,2600640.0]|9999 |
|[4.2,63.0,2.0,4000.0,6.400000095367432,28.0,10.0,32.0,15.0,1123200.0]  |9990 |
|[4.3,75.0,4.0,5000.0,6.599999904632568,58.0,12.0,64.0,25.0,2600640.0]  |11999|
|[4.1,73.0,4.0,6000.0,6.400000095367432,61.0,12.0,64.0,0.0,1152000.0]   |11999|
|[4.1,69.0,4.0,5000.0,6.5,22.0,11.0,64.0,15.0,1152000.0]                |11999|
|[4.4,75.0,6.0,5000.0,6.5,57.0,12.0,128.0,15.0,1152000.0]               |11990|
|[4.1,76.0,4.0,6000.0,6.400000095367432,81.0,11.0,64.0,15.0,2527200.0]  |11990|
|[4.1,71.0,4.0,5000.0,6.5,56.0,10.0,64.0,15.0,1123200.0]                |11990|
|[4.05,75.0,4.0,5000.0,6.599999904632568,67.0,13.0,64.0,15.0,2600640.0] |11599|
|[4.5,75.0,6.0,6000.0,6.599999904632568,

In [52]:
features_df_final.count()

819

### Training df and testing df split:

In [53]:
features_df_final_train, features_df_final_test = features_df_final.randomSplit([ 0.80, 0.20 ])

In [58]:
print( "Training data records count: ", features_df_final_train.count() )
print( "Testing data records count: ", features_df_final_test.count() )

Training data records count:  651
Testing data records count:  168


In [51]:
from pyspark.ml.regression import LinearRegression

In [63]:
mobile_price_regression_model = LinearRegression( featuresCol="independent_features", labelCol="Price" )
mobile_price_regression_model = mobile_price_regression_model.fit( features_df_final_train )

### Regression model stats and metrics:
-----------------------------------------

In [64]:
# coefficients
mobile_price_regression_model.coefficients

DenseVector([-3743.2034, -8.3817, 1924.6846, -2.5837, 12369.743, 15.3705, -2040.0394, 69.6276, 31.9773, 0.0047])

In [65]:
# intercept
mobile_price_regression_model.intercept

-40455.4168659091

In [68]:
mobile_price_regression_model.epsilon

Param(parent='LinearRegression_4c5afd08080b', name='epsilon', doc='The shape parameter to control the amount of robustness. Must be > 1.0. Only valid when loss is huber')

## Predicting values using the regression model:
--------------------------------------------------

In [66]:
pred_results = mobile_price_regression_model.evaluate( features_df_final_test )

In [67]:
pred_results.predictions.show()

+--------------------+-----+------------------+
|independent_features|Price|        prediction|
+--------------------+-----+------------------+
|[3.95,74.0,4.0,50...|11490|12698.648488800674|
|[4.0,71.0,4.0,500...|12990| 3842.976119983614|
|[4.0,75.0,6.0,600...| 8490|17739.189241929744|
|[4.0,79.0,6.0,500...|17999|19002.905524290676|
|[4.0,80.0,6.0,400...|22999| 24788.77715900266|
|[4.0,80.0,6.0,500...|23990|22512.331120345632|
|[4.0,80.0,6.0,500...|15323|19102.645908551087|
|[4.0,81.0,8.0,500...|12990|23249.807592401958|
|[4.0,84.0,8.0,500...|18708|23744.953987887406|
|[4.0,87.0,8.0,600...|37999|24207.541464262817|
|[4.0,89.0,12.0,50...|34990| 65679.66572310877|
|[4.05,69.0,4.0,50...|10199| 7310.621846252128|
|[4.05,70.0,4.0,50...| 8990|1618.3643590261636|
|[4.05,75.0,4.0,50...|11599|11757.526978282476|
|[4.05,75.0,6.0,50...| 8890| 17486.73748990873|
|[4.05,76.0,4.0,50...| 9799|18358.426431910906|
|[4.05,77.0,6.0,50...|15499|11982.117936425719|
|[4.05,77.0,6.0,50...| 9499| 21562.72602

In [69]:
pred_results.meanAbsoluteError

7722.991994624923

In [70]:
pred_results.meanSquaredError

148299520.2273663

In [71]:
pred_results.r2

0.5838853778585431